In [50]:
import usb.core
import usb.util
from pathlib import Path

# Agilent / Keysight USB-ECal module (adjust if yours differs)
VID, PID = 0x0957, 0x0001

# Device-specific control requests (from the Facedancer emulator)
REQ_SET_PTR   = 0x02          # vendor OUT: set EEPROM read pointer
REQ_FINISH    = 0x04          # vendor OUT: optional “done”
BMRT_VENDOR_OUT = 0x40        # 0b01_00_0000  (Host→Dev | Vendor | Device)

BLOCK         = 0x20          # 32 bytes per read
EEPROM_BYTES  = 0x400         # 1 KiB total

TIMEOUT = 10                 # Timeout in ms


In [51]:
def find_ecal():
    """Locate the first matching USB device and claim its interface."""
    dev = usb.core.find(idVendor=VID, idProduct=PID)
    if dev is None:
        raise RuntimeError("ECal module not found (check VID/PID or driver).")

    # Detach any kernel driver (Linux/macOS) so libusb can talk to it
    # if dev.is_kernel_driver_active(0):
        # dev.detach_kernel_driver(0)

    dev.set_configuration()
    return dev


def get_bulk_in_ep(dev):
    """Return endpoint object for bulk-IN endpoint 1 (0x81)."""
    cfg  = dev.get_active_configuration()
    intf = cfg[(0, 0)]
    ep   = usb.util.find_descriptor(
        intf,
        # We want endpoint 1 with IN direction
        custom_match=lambda e: usb.util.endpoint_direction(e.bEndpointAddress)
                               == usb.util.ENDPOINT_IN
                               and (e.bEndpointAddress & 0x0F) == 1
    )
    if ep is None:
        raise RuntimeError("Bulk-IN endpoint 1 not found.")
    return ep


def set_pointer(dev, startAddress, addr):
    """Move internal read pointer to <addr>."""
    wValue = startAddress + EEPROM_BYTES - addr       # emulator logic: addr = 0x400 – wValue
    print("Requesting address: " + str(wValue))
    dev.ctrl_transfer(
        BMRT_VENDOR_OUT, REQ_SET_PTR,
        wValue=wValue, wIndex=0, data_or_wLength=None, timeout=TIMEOUT
    )


In [52]:
def dump_eeprom(startAddress, out_file: Path = Path("ecal_eeprom.bin")) -> Path:
    dev = find_ecal()
    ep  = get_bulk_in_ep(dev)

    data = bytearray()

    for offset in range(0, EEPROM_BYTES, BLOCK):
        set_pointer(dev, startAddress, offset)
        data.extend(ep.read(BLOCK, timeout=10))

    # Optional “we’re done” request (ignored by some modules)
    try:
        dev.ctrl_transfer(BMRT_VENDOR_OUT, REQ_FINISH, 0, 0, None, 10)
    except usb.core.USBError:
        pass

    out_file.write_bytes(data)
    print(f"Wrote {len(data)} bytes -> {out_file.resolve()}")
    return out_file


In [54]:
bin_path = dump_eeprom(0, Path("ecal_eeprom_testB_1.bin"))
first = bin_path.read_bytes()[:64]
print(first.hex(" ", 1))

bin_path = dump_eeprom(0x0400, Path("ecal_eeprom_testB_2.bin"))
first = bin_path.read_bytes()[:64]
print(first.hex(" ", 1))

bin_path = dump_eeprom(0x0800, Path("ecal_eeprom_testB_3.bin"))
first = bin_path.read_bytes()[:64]
print(first.hex(" ", 1))

bin_path = dump_eeprom(0x0c00, Path("ecal_eeprom_testB_4.bin"))
first = bin_path.read_bytes()[:64]
print(first.hex(" ", 1))

bin_path = dump_eeprom(0x1000, Path("ecal_eeprom_testB_5.bin"))
first = bin_path.read_bytes()[:64]
print(first.hex(" ", 1))

bin_path = dump_eeprom(0x1400, Path("ecal_eeprom_testB_6.bin"))
first = bin_path.read_bytes()[:64]
print(first.hex(" ", 1))

bin_path = dump_eeprom(0x1800, Path("ecal_eeprom_testB_7.bin"))
first = bin_path.read_bytes()[:64]
print(first.hex(" ", 1))

bin_path = dump_eeprom(0x1c00, Path("ecal_eeprom_testB_8.bin"))
first = bin_path.read_bytes()[:64]
print(first.hex(" ", 1))

bin_path = dump_eeprom(0x2000, Path("ecal_eeprom_testB_9.bin"))
first = bin_path.read_bytes()[:64]
print(first.hex(" ", 1))


# Quick hex preview (first 64 bytes)
first = bin_path.read_bytes()[:64]
print(first.hex(" ", 1))


Requesting address: 1024
Requesting address: 992
Requesting address: 960
Requesting address: 928
Requesting address: 896
Requesting address: 864
Requesting address: 832
Requesting address: 800
Requesting address: 768
Requesting address: 736
Requesting address: 704
Requesting address: 672
Requesting address: 640
Requesting address: 608
Requesting address: 576
Requesting address: 544
Requesting address: 512
Requesting address: 480
Requesting address: 448
Requesting address: 416
Requesting address: 384
Requesting address: 352
Requesting address: 320
Requesting address: 288
Requesting address: 256
Requesting address: 224
Requesting address: 192
Requesting address: 160
Requesting address: 128
Requesting address: 96
Requesting address: 64
Requesting address: 32
Wrote 1024 bytes -> C:\Users\hendo\OneDrive - rhconsulting.co.nz\Documents\Jupyter Notebooks\SNA5k\N4691-60004 eCal\ecal_eeprom_testB_1.bin
48 50 38 35 30 36 30 43 20 45 43 41 4c 00 ff ff ff ff ff ff 64 00 4e 6f 76 20 32 38 20 31 39 3

In [23]:
import usb.backend.libusb1 as lib1; 

print(lib1.get_backend())

In [5]:
import usb.core, usb.util, usb.backend.libusb1 as lib1, platform, sys, ctypes, shutil, os
backend = lib1.get_backend()
print(f"Python: {platform.python_version()}  ({platform.architecture()[0]})")
print(f"libusb backend: {backend or '❌ NOT FOUND'}")

# List every Agilent/Keysight USB device PyUSB can see
for d in usb.core.find(find_all=True, idVendor=0x0957):
    try:
        # string descriptors are optional; ignore failures
        mfg  = usb.util.get_string(d, d.iManufacturer)
        prod = usb.util.get_string(d, d.iProduct)
    except Exception:
        mfg = prod = "?"
    print(f"Found 0x0957:0x{d.idProduct:04x}  {mfg} – {prod}")


Python: 3.12.6  (64bit)
libusb backend: <usb.backend.libusb1._LibUSB object at 0x0000020F46866E10>
Found 0x0957:0x0107  ? – ?
Found 0x0957:0x0001  Agilent Technologies – USB ECal Module
